# Add four numbers

In [1]:
import numpy as np
import tensorflow as tf

2023-10-13 17:31:09.934155: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
"""
Generates a simple TfLite model that adds 4 numbers.

Basic Usage:

  model = generate_model(False)

Usage where you want model written to file:

  file_path = "some file path"
  model = generate_model(True, file_path)
"""

class AddFourNumbers(tf.Module):
  @tf.function(
      input_signature=[
          tf.TensorSpec(shape=[1], dtype=tf.float32, name="a"),
          tf.TensorSpec(shape=[1], dtype=tf.float32, name="b"),
          tf.TensorSpec(shape=[1], dtype=tf.float32, name="c"),
          tf.TensorSpec(shape=[1], dtype=tf.float32, name="d"),
      ]
  )
  def __call__(self, a, b, c, d):
    return a + b + c + d


In [3]:
def get_model_from_concrete_function():
  """Accumulator model built via TF concrete functions."""
  model = AddFourNumbers("AddFourNumbers")
  concrete_func = model.__call__.get_concrete_function()
  converter = tf.lite.TFLiteConverter.from_concrete_functions(
      [concrete_func], model
  )
  return converter.convert()


In [8]:
def generate_model(write_file=True, filename="models/add.tflite"):
  model = get_model_from_concrete_function()
  if write_file:
    with open(filename, "wb") as f:
      f.write(model)
  return model

In [9]:
model = generate_model()

INFO:tensorflow:Assets written to: /tmp/tmpjvh_bl4j/assets


INFO:tensorflow:Assets written to: /tmp/tmpjvh_bl4j/assets
2023-10-13 17:37:17.462646: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-10-13 17:37:17.462673: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-10-13 17:37:17.463120: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpjvh_bl4j
2023-10-13 17:37:17.463346: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2023-10-13 17:37:17.463361: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /tmp/tmpjvh_bl4j
2023-10-13 17:37:17.463957: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2023-10-13 17:37:17.470293: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: /tmp/tmpjvh_bl4j
2023-10-13 17:37:17.477198: I tensorflow/cc/saved_model/loader.cc:314] SavedModel

In [11]:
model_path = "models/add.tflite"
tflite_interpreter = tf.lite.Interpreter(
      model_path=model_path,
      experimental_op_resolver_type=tf.lite.experimental.OpResolverType.
      BUILTIN_REF,
  )
tflite_interpreter.allocate_tensors()

In [12]:
input_details = tflite_interpreter.get_input_details()
print(input_details)

[{'name': 'serving_default_d:0', 'index': 0, 'shape': array([1], dtype=int32), 'shape_signature': array([1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'serving_default_c:0', 'index': 1, 'shape': array([1], dtype=int32), 'shape_signature': array([1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'serving_default_b:0', 'index': 2, 'shape': array([1], dtype=int32), 'shape_signature': array([1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsi

In [14]:
print("input shape: ",input_details[0]['shape'])
print("input dtype: ",input_details[0]['dtype'])

input shape:  [1]
input dtype:  <class 'numpy.float32'>


In [33]:
output_details = tflite_interpreter.get_output_details()
print(output_details)
print('index :',output_details[0]['index'])

[{'name': 'PartitionedCall:0', 'index': 6, 'shape': array([1], dtype=int32), 'shape_signature': array([1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
6


In [46]:
tflite_interpreter.set_tensor(0,np.array([3],dtype=np.float32))
tflite_interpreter.set_tensor(1,np.array([4],dtype=np.float32))
tflite_interpreter.set_tensor(2,np.array([3],dtype=np.float32))
tflite_interpreter.set_tensor(3,np.array([10],dtype=np.float32))

In [47]:
tflite_interpreter.invoke()

In [48]:
result = tflite_interpreter.get_tensor(output_details[0]['index'])
print("result: ",result)

result:  [20.]
